Detailed Editor
  User needs to set userkey
  
  Can be used to select and edit more detailed properties of threat events

In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
import psycopg2
from IPython.display import display, Markdown
import qual
import quant

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)

cursor.execute ('SELECT name, mission, org FROM orgv')
orgs = cursor.fetchall()

cursor.execute('SELECT asset, kind, name, description FROM assetv ORDER BY kind, name ASC')
rows = cursor.fetchall()
  
threats = pd.read_sql_query ('SELECT threat_id, source_id, impact_id, name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init, vdist, vparam2, pdist, pparam2, cap_dist, cap_param2, init_dist, init_param2, impact_dist, impact_param2 FROM detailv WHERE init>0', conn)


In [ ]:
for i in threats.index:
    v = qual.qdict[qual.norm(threats.at[i,'vulnerability'])]
    l = qual.qdict[qual.norm(threats.at[i,'init'])]
    im = qual.qdict[qual.norm(threats.at[i,'impact'])]
    threats.at[i,'vuln'] = v
    threats.at[i,'imp'] = im
    threats.at[i,'cap'] = qual.qdict[qual.norm(threats.at[i,'capacity'])]
    threats.at[i,'likely'] = l
    o = qual.overall[l][v]
    threats.at[i,'overall_likelihood'] = o
    threats.at[i,'risk_level'] = qual.risk_level[o][im]



In [ ]:
def printmd(string):
    display(Markdown(string))
    
def printcell (name, description, vuln, imp, cap, overall, vulnerability, vulnerabilityd, vulnerability2, pervasive, impact, impactd, impact2, impact_name, impact_desc, init, initd, init2, capacity, capacityd, capacity2):
    str = "<table><tr>"
    str += "<td><table><tr><td></td><td>Rating</td><td>Distribution</td><td>Value</td><td>Value 2</td></tr>"
    str += "<tr><td><b>Vulnerability:</b></td><td>" + vuln + "</td><td>" + quant.prob_rdict[vulnerabilityd] + "</td><td>" + vulnerability + "</td><td>" + vulnerability2 + "</td></tr>"
    str += "<tr><td><b>Capacity:</b></td><td>" + cap + "</td><td>" + quant.prob_rdict[capacityd] + "</td><td>" + capacity + "</td><td>" + capacity2 +  "</td></tr>"
    str += "<tr><td><b>Impact:</b></td><td>" + imp + "</td><td>" + quant.prob_rdict[impactd] + "</td><td>" + impact + "</td><td>" + impact2 + "</td></tr>"
    str += "<tr><td><b>Likelihood:</b></td><td>" + overall + "</td><td>" + quant.prob_rdict[initd] + "</td><td>" + init + "</td><td>" + init2 + "</td></tr>"
    str += "</table></td>"
    str += "<td><table><tr><td><b>Name:</b></td><td valign='left'>" + name + "</td></tr>"
    str += "<tr><td><b>Description:</b></td><td width='200px'>" + description + "</td></tr>"
    str += "<tr><td><b>Impact:</b></td><td valign='left'>" + impact_name + "</td></tr>"
    str += "<tr><td><b>Description:</b></td><td width='200px'>" + impact_desc + "</td></tr>"
    str += "</table></td>"
    str += "</tr></table>"
    return str

labels=['Proc name:', 'Proc name:', 'System name:', 'System name:', 'System name:', 'Asset name:', 'Asset name:', 'Asset name:', 'Asset name:', 'Asset name:']

orghtml = "<table><tr><td colspan='2'><h1>" + orgs[0][0] + "</h1></td></tr>"
orghtml += "<tr><td><h2>Mission</h2></td><td>" + orgs[0][1] + "</td></tr>"
for x in range(0,10):
    orghtml += "<tr><td><b>" + labels[x] + "</b></td><td>" + rows[x][2] + "</td>"
    orghtml += "<td>" + rows[x][3] + "</td>"
    orghtml += "</tr>"
orghtml += "</table>"

printmd (orghtml)

for i in reversed(qual.qscale):
    printmd ("<h1>" + i + "</h1>")
    tr = threats.loc[threats['risk_level']==i]
    shape = tr.shape

    for j in tr.index:
        s = printcell (tr.at[j,'name'], tr.at[j,'description'], tr.at[j,'vuln'], tr.at[j,'imp'], tr.at[j,'cap'], tr.at[j,'overall_likelihood'], str (tr.at[j,'vulnerability']), tr.at[j,'vdist'], str(tr.at[j,'vparam2']), tr.at[j,'pervasive'], str(tr.at[j,'impact']), tr.at[j,'impact_dist'], str(tr.at[j,'impact_param2']), tr.at[j,'impact_name'], tr.at[j,'impact_description'], str(tr.at[j,'init']), tr.at[j,'init_dist'], str(tr.at[j,'init_param2']), str(tr.at[j,'capacity']), tr.at[j,'cap_dist'], str(tr.at[j,'cap_param2']))
        printmd (s)
        
        sim_runs = 100

        event_loss = tr.at[j,'impact']
        loss_var = tr.at[j,'impact_param2']
        loss_fn = quant.prob_fn[quant.prob_rdict[tr.at[j,'impact_dist']]]

        af = tr.at[j,'init']
        attack_freq = af / 10
        source_cap = tr.at[j,'capacity']
        source_var = tr.at[j,'cap_param2']
        cap_fn = quant.prob_fn[quant.prob_rdict[tr.at[j,'cap_dist']]]

        vuln_score = tr.at[j,'vulnerability']
        vuln_var = tr.at[j,'vparam2']
        vuln_fn = quant.prob_fn[quant.prob_rdict[tr.at[j,'vdist']]]


        attack_chance = quant.prob_fn[quant.prob_rdict[tr.at[j,'init_dist']]](attack_freq, tr.at[j,'init_param2'], sim_runs)
        cap_chance = cap_fn(source_cap, source_var, sim_runs)
        vuln_chance = vuln_fn(vuln_score, vuln_var, sim_runs)

        count, bins, ignored = plt.hist(vuln_chance, 30, density=True)
        plt.plot ([vuln_score, vuln_score], [0, 0.02], linewidth=2, color='r')
        plt.title ("Vulnerability")
        plt.show()

        data = plt.hist(cap_chance, 30, density=True)
        plt.plot ([source_cap, source_cap], [0, 0.02], linewidth=2, color='r')
        plt.title("Source Capacity")
        plt.show()

        data = plt.hist(attack_chance, 30, density=True)
        plt.title(str(attack_freq) + " attacks expected annually")
        plt.show()

        loss_list = []
        loss_sum = 0
        loss_max = 0

        for ys in attack_chance:
            if ys > 0:
                loss = 0
                count = int (ys)
                c = cap_fn (source_cap, source_var, count)
                v = vuln_fn (vuln_score, vuln_var, count)
                l = loss_fn (event_loss, loss_var, count)
                for i in range(1, count):
                    if (c[i] >= v[i] ):
                        loss = loss + l[i]
                loss_list.append (loss)
                if loss > loss_max:
                    loss_max = loss
                loss_sum += loss
     
        plt.hist(loss_list, 10, density=True)
        plt.title ("Expected annual loss")
        plt.show()

        print ("max loss is " + str(int(loss_max)))
        print ("average loss is " + str(int(loss_sum/len(loss_list))))
        